<a href="https://colab.research.google.com/github/calvinsuzuki/CNN_lab/blob/main/cnn_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download de arquivos de dataset

In [5]:
!rm -r sample_data
!rm -r dataset
!gdown 1HDCqyHz6hCUvQ3xdktqRE9tht8wOOY80
!unzip -qq ./dataset.zip -d .

rm: cannot remove 'sample_data': No such file or directory
rm: cannot remove 'dataset': No such file or directory
Downloading...
From (original): https://drive.google.com/uc?id=1HDCqyHz6hCUvQ3xdktqRE9tht8wOOY80
From (redirected): https://drive.google.com/uc?id=1HDCqyHz6hCUvQ3xdktqRE9tht8wOOY80&confirm=t&uuid=623cd47d-9707-4cc9-a131-0c1da6642ae4
To: /content/dataset.zip
100% 41.4M/41.4M [00:00<00:00, 163MB/s]


In [6]:
import tensorflow as tf

## Lendo o dataset de gatos e cães

In [16]:
import os
import cv2
import numpy as np

# Cats and dogs dataset
train_dir = './dataset/training_set'
images_dir = sorted(os.listdir(os.path.join(train_dir, 'images')))
images_path = [os.path.join(train_dir, 'images', img) for img in images_dir]
labels_dir = sorted(os.listdir(os.path.join(train_dir, 'labels')))
labels_path = [os.path.join(train_dir, 'labels', lbl) for lbl in labels_dir]

# Read X and Y
X = [cv2.imread(img_path) for img_path in images_path]
Y = ['label'] * len(X)
for i, lbl_path in enumerate(labels_path):
    with open(lbl_path, 'r') as file:
        label = file.read().strip()
        Y[i] = 1 if label == 'cat' else 0
X = np.array(X) / 255
Y = np.array(Y) / 255

## Definindo o modelo
Uma rede neural com duas camadas densas de 128 neurônios cada, ativação ReLU e uma saída sigmoide.

In [17]:
# Define the model
# Define the model with convolutional layers
model = tf.keras.models.Sequential([
    # Convolutional layer with 32 filters, 3x3 kernel, and ReLU activation
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),

    # MaxPooling layer to reduce spatial dimensions
    tf.keras.layers.MaxPooling2D((2, 2)),

    # Second convolutional layer with 64 filters
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),

    # MaxPooling layer to reduce spatial dimensions
    tf.keras.layers.MaxPooling2D((2, 2)),

    # Third convolutional layer with 128 filters
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),

    # MaxPooling layer to reduce spatial dimensions
    tf.keras.layers.MaxPooling2D((2, 2)),

    # Flatten the output to connect to a dense layer
    tf.keras.layers.Flatten(),

    # Fully connected layer with 128 units and ReLU activation
    tf.keras.layers.Dense(64, activation='relu'),

    # Output layer with a single unit for binary classification (sigmoid activation)
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Display the model summary
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │     1,605,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,699,009 (6.48 MB)

 Trainable params: 1,699,009 (6.48 MB)

 Non-trainable params: 0 (0.00 B)

## Compilando e treinando o modelo
O modelo é treinado por 100 épocas no mesmo conjunto de dados pequeno.

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(X.shape)
print(Y.shape)
model.fit(X, Y, epochs=10, batch_size=128)

(4000, 128, 128, 3)
(4000,)
Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 133s 4s/step - accuracy: 0.4963 - loss: 0.1034
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 141s 4s/step - accuracy: 0.5032 - loss: 0.0149
Epoch 3/10


## Avaliando o modelo
Como os dados de treino e teste são os mesmos, esperamos uma acurácia muito alta (overfitting).

In [ ]:
loss, acc = model.evaluate(X, y)
print("Acurácia no treino:", acc)